In [ ]:
# ============================================================
# 📦 Calculadora de LEF (Lote Econômico de Fabricação) — Colab Ready
# Versão enxuta: 5 indicadores + explicações
# ============================================================

# --- Setup automático para Colab (instala/ativa widgets) ---
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "ipywidgets"], check=True)
    from google.colab import output
    output.enable_custom_widget_manager()
else:
    print("Executando fora do Colab. Se faltar ipywidgets, use fallback_cli().")

# --- Imports principais ---
import math
import pandas as pd
from dataclasses import dataclass

# -----------------------------
# Utilitários de formatação BR
# -----------------------------
def fmt_br(valor: float, casas: int = 2) -> str:
    """Formata com milhar '.' e decimal ','."""
    s = f"{valor:,.{casas}f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")

def fmt_int_br(valor: float) -> str:
    """Inteiro com separador de milhar '.'."""
    return fmt_br(round(valor), 0)

def fmt_brl(valor: float) -> str:
    return f"R$ {fmt_br(valor, 2)}"

# -----------------------------
# Modelos de dados
# -----------------------------
@dataclass
class EntradasLEF:
    D: float      # Demanda por período (unid/tempo)
    P: float      # Taxa de produção por período (unid/tempo) — deve ser > D
    t: float      # Taxa de manutenção/armazenagem no período (ex.: 0,30 = 30%/ano)
    Cunit: float  # Custo unitário (R$/unid)
    Cp: float     # Custo de setup por lote (R$)

@dataclass
class SaidasLEF:
    LEF: float         # unidades por lote (ótimo)
    N_lotes: float     # número de lotes por período
    C_setup: float     # custo de setup no período
    C_estoque: float   # custo de manutenção no período
    CT: float          # custo total no período

# -----------------------------
# Explicações (HTML)
# -----------------------------
def explicacao_entradas_html():
    from IPython.display import HTML
    html = """
    <div style="border:1px solid #ddd;padding:12px;border-radius:8px;margin:6px 0">
      <b>O que preencher:</b>
      <ul>
        <li><b>D (Demanda por período)</b>: consumo/entrega média no período (ex.: <i>3.600.000 unid/ano</i>).</li>
        <li><b>P (Produção por período)</b>: capacidade de fabricação no mesmo período de D (ex.: <i>7.200.000 unid/ano</i>). <u>Precisa ser maior que D</u>.</li>
        <li><b>t (Taxa de manutenção)</b>: custo percentual para manter estoque no período (ex.: <i>0,30</i> = 30%/ano).</li>
        <li><b>Cunit (Custo unitário)</b>: custo de produção por unidade (ex.: <i>0,35</i> = R$ 0,35/unid).</li>
        <li><b>Cp (Setup por lote)</b>: custo/troca de lote (ex.: <i>450</i> = R$ 450 por setup).</li>
      </ul>
      <b>Exemplo rápido:</b> D=3.600.000; P=7.200.000; t=0,30; Cunit=0,35; Cp=450 → entradas anuais coerentes.
      <br><i>Checklist:</i> D e P na mesma base de tempo; P &gt; D; t em fração; Cunit e Cp em R$.
    </div>
    """
    return HTML(html)

def explicacao_itens_html():
    from IPython.display import HTML
    html = """
    <div style="border:1px solid #ddd;padding:12px;border-radius:8px;margin:6px 0">
      <b>O que significam os 5 itens:</b>
      <ul>
        <li><b>LEF (unid./lote)</b>: tamanho ótimo do lote para minimizar custos.</li>
        <li><b>Nº de lotes por período</b>: quantas vezes você fabrica no período.</li>
        <li><b>Custo de setup (período)</b>: custo total das trocas de lote.</li>
        <li><b>Custo de manutenção (período)</b>: custo de manter o estoque médio que se forma ao produzir.</li>
        <li><b>Custo total (período)</b>: soma de setup + manutenção (mínimo na condição ótima).</li>
      </ul>
      <i>Na prática:</i> programe a produção em lotes de tamanho LEF e ajuste a frequência (nº de lotes)
      conforme a capacidade e as janelas de entrega, vigiando t e Cp para oportunidades de redução.
    </div>
    """
    return HTML(html)

def conclusoes_praticas(s: SaidasLEF):
    from IPython.display import HTML
    frases = [
        f"Lote econômico recomendado: <b>{fmt_int_br(s.LEF)}</b> peças.",
        f"Número de lotes no período: <b>{fmt_br(s.N_lotes, 2)}</b>.",
        f"Custo de setup no período: <b>{fmt_brl(s.C_setup)}</b>.",
        f"Custo de manutenção no período: <b>{fmt_brl(s.C_estoque)}</b>.",
        f"Custo total projetado no período: <b>{fmt_brl(s.CT)}</b>."
    ]
    itens = "".join([f"<li>{f}</li>" for f in frases])
    return HTML(f"<div style='margin:6px 0'><ul>{itens}</ul></div>")

# -----------------------------
# Validação e cálculo
# -----------------------------
def validar(e: EntradasLEF):
    if e.D <= 0 or e.P <= 0:
        raise ValueError("D e P devem ser positivos.")
    if e.t <= 0 or e.Cunit <= 0 or e.Cp <= 0:
        raise ValueError("t, Cunit e Cp devem ser positivos.")
    if e.P <= e.D:
        raise ValueError("P deve ser MAIOR que D para a aplicação do LEF (reposição progressiva).")

def calcular_lef(e: EntradasLEF) -> SaidasLEF:
    validar(e)
    fator = 1.0 - e.D / e.P               # (1 - D/P)
    LEF = math.sqrt((2.0 * e.Cp * e.D) / (e.t * e.Cunit * fator))
    Em = 0.5 * LEF * fator                # estoque médio (para custo de manutenção)
    N_lotes = e.D / LEF
    C_estoque = e.t * e.Cunit * Em
    C_setup   = e.Cp * (e.D / LEF)
    CT = C_estoque + C_setup
    return SaidasLEF(LEF=LEF, N_lotes=N_lotes, C_setup=C_setup, C_estoque=C_estoque, CT=CT)

# -----------------------------
# Tabela dos 5 indicadores (renomeada p/ gerar_tabela)
# -----------------------------
def gerar_tabela(s: SaidasLEF) -> pd.DataFrame:
    data = [
        ("LEF (unid./lote)",              fmt_int_br(s.LEF)),
        ("Nº de lotes por período",       fmt_br(s.N_lotes, 2)),
        ("Custo de setup (período)",      fmt_brl(s.C_setup)),
        ("Custo de manutenção (período)", fmt_brl(s.C_estoque)),
        ("Custo total (período)",         fmt_brl(s.CT)),
    ]
    return pd.DataFrame(data, columns=["Indicador", "Valor"])

# -----------------------------
# Exemplo validado (caso clássico)
# -----------------------------
def exemplo():
    e = EntradasLEF(D=3_600_000, P=7_200_000, t=0.30, Cunit=0.35, Cp=450.0)
    s = calcular_lef(e)
    print("LEF calculado (esperado ≈ 248.423 unid):", fmt_int_br(s.LEF))
    df = gerar_tabela(s)
    try:
        from IPython.display import display
        display(df)
        display(conclusoes_praticas(s))
        display(explicacao_itens_html())
    except Exception:
        print(df.to_string(index=False))
    return e, s, df

# -----------------------------
# UI com ipywidgets (com textos explicativos)
# -----------------------------
def ui():
    try:
        import ipywidgets as widgets
        from IPython.display import display, clear_output
    except Exception:
        print("⚠️ Widgets indisponíveis. Usando modo texto (fallback_cli).")
        fallback_cli()
        return

    D     = widgets.FloatText(description="Demanda (D)",  value=3_600_000.0)
    P     = widgets.FloatText(description="Produção (P)", value=7_200_000.0)
    t     = widgets.FloatText(description="Taxa t",       value=0.30)
    Cunit = widgets.FloatText(description="Custo Cunit",  value=0.35)
    Cp    = widgets.FloatText(description="Setup Cp",     value=450.0)

    bt = widgets.Button(description="Calcular LEF + Tabela")
    status = widgets.HTML("")
    out = widgets.Output()

    # Bloco explicativo antes do form
    display(explicacao_entradas_html())

    def on_click(_):
        with out:
            clear_output(wait=True)
            try:
                e = EntradasLEF(D.value, P.value, t.value, Cunit.value, Cp.value)
                s = calcular_lef(e)
                df = gerar_tabela(s)
                display(df)
                display(conclusoes_praticas(s))      # resumo prático com valores
                display(explicacao_itens_html())      # explicação do que é cada item
                bt.button_style = "success"
                bt.description = "Recalcular ✓"
                status.value = "<span style='color:green'>Cálculo concluído.</span>"
            except Exception as exc:
                bt.button_style = "danger"
                status.value = f"<span style='color:red'>Erro: {exc}</span>"

    bt.on_click(on_click)

    display(
        widgets.VBox([
            widgets.HTML("<h3>Calculadora de LEF — resultados essenciais</h3>"),
            widgets.HBox([D, P]),
            widgets.HBox([t, Cunit, Cp]),
            widgets.HBox([bt, status]),
            out
        ])
    )

# -----------------------------
# Fallback em modo texto
# -----------------------------
def fallback_cli():
    print("=== Calculadora LEF (modo texto) ===")
    D     = float(input("Demanda D: ").strip().replace(',','.'))
    P     = float(input("Produção P (deve ser > D): ").strip().replace(',','.'))
    t     = float(input("Taxa de manutenção t (ex.: 0.30): ").strip().replace(',','.'))
    Cunit = float(input("Custo unitário Cunit (R$): ").strip().replace(',','.'))
    Cp    = float(input("Custo de setup Cp (R$): ").strip().replace(',','.'))
    e = EntradasLEF(D, P, t, Cunit, Cp)
    s = calcular_lef(e)
    df = gerar_tabela(s)
    print("\n--- Tabela ---")
    print(df.to_string(index=False))
    print("\nResumo prático:")
    for linha in [
        f"Lote econômico recomendado: {fmt_int_br(s.LEF)} peças.",
        f"Número de lotes no período: {fmt_br(s.N_lotes, 2)}.",
        f"Custo de setup no período: {fmt_brl(s.C_setup)}.",
        f"Custo de manutenção no período: {fmt_brl(s.C_estoque)}.",
        f"Custo total projetado no período: {fmt_brl(s.CT)}."
    ]:
        print("-", linha)

# -----------------------------
# Inicialização padrão (abre UI)
# -----------------------------
try:
    ui()
except Exception:
    print("Abrindo fallback CLI...")
    fallback_cli()

# Dicas:
# - Para validar com o caso clássico, rode:  exemplo()
# - Lembre-se: D e P devem estar na MESMA base de tempo (por dia, mês ou ano).
